In [1]:
import os
os.chdir('C:/Users/RH/A_whole_try_prog/ESM2560_clean/data')

In [2]:
import pandas as pd
import numpy as np
import esm
import torch

print(pd.__version__)
print(np.__version__)
print(esm.__version__)
print(torch.__version__)

1.5.3
1.22.0
2.0.0
2.0.0


In [3]:
import numpy as np
import pandas as pd

In [4]:
#training dataset loading
#打开指定路径的excel文件
excel_path = r'C:\Users\RH\A_whole_try_prog\Sequence_with_random_6v3.xlsx'
dataset = pd.read_excel(excel_path, na_filter=False)
sequence_list = dataset['Sequence']

In [5]:
print(sequence_list)

0            VKV
1           PKAV
2            IKL
3          VPSGK
4            EAK
          ...   
1079     DSWLNLG
1080          SY
1081          CT
1082    GIDNEPLT
1083        AQSD
Name: Sequence, Length: 1084, dtype: object


In [6]:
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence)
print(peptide_sequence_list)

[('VKV', 'VKV'), ('PKAV', 'PKAV'), ('IKL', 'IKL'), ('VPSGK', 'VPSGK'), ('EAK', 'EAK'), ('FVAGK', 'FVAGK'), ('KAI', 'KAI'), ('KVI', 'KVI'), ('KD', 'KD'), ('LQPGQGQQ', 'LQPGQGQQ'), ('YFYPEL', 'YFYPEL'), ('FYPEL', 'FYPEL'), ('YPEL', 'YPEL'), ('PEL', 'PEL'), ('EL', 'EL'), ('LMSYMWSTSM', 'LMSYMWSTSM'), ('LELHKLRSSHWFSRR', 'LELHKLRSSHWFSRR'), ('PSHDAHPE', 'PSHDAHPE'), ('SHDAHPE', 'SHDAHPE'), ('VDHDHPE', 'VDHDHPE'), ('PKAVHE', 'PKAVHE'), ('PAGY', 'PAGY'), ('PHHADS', 'PHHADS'), ('VDYP', 'VDYP'), ('LQPGQGQQG', 'LQPGQGQQG'), ('AQIPQQ', 'AQIPQQ'), ('YAEERYPIL', 'YAEERYPIL'), ('WYSLAMAASDI', 'WYSLAMAASDI'), ('VAPFPGVFGK', 'VAPFPGVFGK'), ('HLPLPL', 'HLPLPL'), ('PPQ', 'PPQ'), ('GPVRGPFPII', 'GPVRGPFPII'), ('NQE', 'NQE'), ('AKYIPIQYVLSRYPSY', 'AKYIPIQYVLSRYPSY'), ('IPIQYVL', 'IPIQYVL'), ('PPK', 'PPK'), ('FDSGPAGVL', 'FDSGPAGVL'), ('NGPLQAGQPGER', 'NGPLQAGQPGER'), ('HGPLGPL', 'HGPLGPL'), ('NADFGLNGLEGLA', 'NADFGLNGLEGLA'), ('NGLEGLK', 'NGLEGLK'), ('LGLNGDDVN', 'LGLNGDDVN'), ('VKAGFAWTANQQLS', 'VKAGFAW

In [7]:
#调试完毕，以此为准！
import torch
import esm
import pandas as pd

#load ESM-2 model
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()
    
#load data
data = peptide_sequence_list
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    
# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[36], return_contacts=True)
token_representations = results["representations"][36]
    
# Generate per-sequence representations via averaging
sequence_representations = []
for i, token_len in enumerate(batch_lens):
    each_seq_rep = token_representations[i, 1:token_len - 1].mean(0).tolist()  
    sequence_representations.append(each_seq_rep)

embedding_results = pd.DataFrame(sequence_representations)
print(embedding_results)
embedding_results.to_csv('ESM_dimension_2560.csv')

          0         1         2         3         4         5         6     \
0     0.101322 -0.054847 -0.076030 -0.143270  0.005912 -0.169835  0.043163   
1     0.017252 -0.119017 -0.060289 -0.150465 -0.020410 -0.183808  0.003710   
2     0.081909 -0.006320 -0.017331  0.009836  0.062259 -0.122803  0.112387   
3     0.007409 -0.046734  0.027885 -0.074698  0.065265 -0.180301 -0.045657   
4     0.012916 -0.036204 -0.076009 -0.109539 -0.008256 -0.169509 -0.053947   
...        ...       ...       ...       ...       ...       ...       ...   
1079  0.038516 -0.062016  0.023377 -0.041475  0.002053 -0.186326 -0.025836   
1080  0.009051 -0.066509  0.035998 -0.030954  0.004116 -0.101101  0.056748   
1081  0.045995 -0.105470  0.010240 -0.080949  0.100331 -0.077811  0.108092   
1082 -0.004965 -0.046012  0.013102 -0.032615  0.027343 -0.182819 -0.031851   
1083  0.021076 -0.100040 -0.032961 -0.065161 -0.057682 -0.221389  0.009935   

          7         8         9     ...      2550      2551    